In [112]:
import cv2
import numpy as np
import os

lp_images_path = 'selection/'
lp_width_px = 300
lp_height_px = 100
lp_images = os.listdir(lp_images_path)

In [113]:
def filtered_lines(lines, is_horizonal):
    filtered = []
    for line in lines:
        for rho, theta in line:
                angle = np.degrees(theta)
                angle_threshold = 5

                if is_horizonal and np.abs(angle - 90) > angle_threshold:
                    continue
                
                if not is_horizonal and (angle > angle_threshold or angle < 180 - angle_threshold):
                    continue

                filtered.append([rho, theta])
    return filtered

In [114]:
def plot_edge_detection(threshold_1, threshold_2 ,min_length_v, max_gap_v, min_length_h, max_gap_h, img):
    img_height, img_width, _ = img.shape
    window_title = 'Canny Edge Detection'
    cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_title, 400, 800)

    prep_1 = img.copy()
    prep_1 = cv2.cvtColor(prep_1, cv2.COLOR_BGR2GRAY)
    rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (200, 100))
    prep_1 = cv2.morphologyEx(prep_1, cv2.MORPH_BLACKHAT, rectKern)
    prep_1 = cv2.equalizeHist(prep_1)
    prep_1 = cv2.GaussianBlur(prep_1, (3, 3), 0)

    # canny edge detection
    canny = cv2.Canny(prep_1, threshold_1, threshold_2, apertureSize=3)

    # dilate and erode to close gaps
    prep_2 = canny.copy()
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 2))
    prep_2 = cv2.morphologyEx(prep_2, cv2.MORPH_CLOSE, kernel)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 5))
    prep_2 = cv2.morphologyEx(prep_2, cv2.MORPH_CLOSE, kernel)

    images = [
        img,
        cv2.cvtColor(prep_1, cv2.COLOR_GRAY2RGB),
        cv2.cvtColor(canny, cv2.COLOR_GRAY2RGB),
        cv2.cvtColor(prep_2, cv2.COLOR_GRAY2RGB),
    ]

    # hough lines
    hough = img.copy()
    vertical_lines = cv2.HoughLines(prep_2, 1, np.pi/180, min_length_v, max_gap_v)
    print(vertical_lines[0])


    vertical_lines = filtered_lines(vertical_lines, False)
    print(vertical_lines)

    horizontal_lines = cv2.HoughLinesP(prep_2, 1, np.pi/180, min_length_h, max_gap_h)
    horizontal_lines = filtered_lines(horizontal_lines, True)

    lines = vertical_lines

    if lines is not None:
        for line in lines:
            for rho, theta in line:
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a * rho
                y0 = b * rho
                x1 = int(x0 + 1000 * (-b))
                y1 = int(y0 + 1000 * (a))
                x2 = int(x0 - 1000 * (-b))
                y2 = int(y0 - 1000 * (a))

                cv2.line(hough, (x1, y1), (x2, y2), (0, 255, 0), 2)
        images.append(hough)

    output_image = np.concatenate(images, axis=0)
    cv2.imshow(window_title, output_image)

In [115]:
threshold_window = 'Thresholds'
cv2.namedWindow(threshold_window, cv2.WINDOW_NORMAL)
cv2.resizeWindow(threshold_window, 400, 100)

img = cv2.imread('selection/18404_0_0_5_29_31_27_7.jpg')

def show_detection(val):
    threshold1 = cv2.getTrackbarPos('Threshold 1', threshold_window)
    threshold2 = cv2.getTrackbarPos('Threshold 2', threshold_window)
    min_length_v = cv2.getTrackbarPos('Min Line Length v', threshold_window)
    max_gap_v = cv2.getTrackbarPos('Max Gap v', threshold_window)
    min_length_h= cv2.getTrackbarPos('Min Line Length h', threshold_window)
    max_gap_h = cv2.getTrackbarPos('Max Gap h', threshold_window)
    plot_edge_detection(threshold1, threshold2, min_length_v, max_gap_v, min_length_h, max_gap_h, img)

cv2.createTrackbar('Threshold 1', threshold_window, 250, 500, show_detection)
cv2.createTrackbar('Threshold 2', threshold_window, 250, 500, show_detection)
cv2.createTrackbar('Min Line Length v', threshold_window, 30, 500, show_detection)
cv2.createTrackbar('Max Gap v', threshold_window, 10, 500, show_detection)
cv2.createTrackbar('Min Line Length h', threshold_window, 30, 500, show_detection)
cv2.createTrackbar('Max Gap h', threshold_window, 10, 500, show_detection)

show_detection(0)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[145.          1.6057029]]


IndexError: list index out of range